In [6]:
with open("input22.txt", "r") as file:
    init_secret = [int(line.strip("\n")) for line in file.readlines()]

In [18]:
#an example
secret_n = 123
for i in range(10):
    secret_n = ((64*secret_n)^secret_n) % 16777216
    secret_n = ((secret_n//32)^secret_n) % 16777216
    secret_n = ((2048*secret_n)^secret_n) % 16777216
    print(secret_n, end=" ")

15887950 16495136 527345 704524 1553684 12683156 11100544 12249484 7753432 5908254 

In [7]:
thesum = 0
for secret_n in init_secret:
    for i in range(2000):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
    thesum += secret_n
thesum

21147129593

Part II

In [1]:
import numpy as np
np.set_printoptions(threshold=10000, linewidth=100000)

with open("input22.txt", "r") as file:
    init_secret = [int(line.strip("\n")) for line in file.readlines()]
# init_secret = [123]

Most efficient solutions: when reading the diffs, make a dictionary with tuples as keys and total sale as values (besides a necessary "last-sale index"), ~27s

In [218]:
prices, totals = [], {}
for j in range(len(init_secret)):
    secret_n = init_secret[j]
    prices.append([secret_n%10])
    for i in range(1,4):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
    for i in range(4,2001):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        thetuple = tuple(prices[-1][ii-4] - prices[-1][ii-5] for ii in range(4))
        if thetuple not in totals: totals[thetuple] = (secret_n%10, j)
        elif totals[thetuple][1] != j: totals[thetuple] = (totals[thetuple][0]+secret_n%10, j)
    if j%100==0: print(f"{j:4d}/{len(init_secret):4d}", end="\r")
max(totals.values()), len(totals), {k:v for k,v in totals.items() if v==(2445, 2459)}

((2445, 2459), 40951, {(0, 0, -2, 2): (2445, 2459)})

The most efficient: dictionary as above, but encoding tuples as four-digit base-20 numbers, ~13s

In [30]:
prices, totals = [], {}
for j in range(len(init_secret)):
    secret_n = init_secret[j]
    prices.append([secret_n%10])
    previous = 0
    for i in range(1,4):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        current = (prices[-1][-1] - prices[-1][-2] + 9)*20**3 + previous//20
        previous = current
    for i in range(4,2001):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        current = (prices[-1][-1] - prices[-1][-2] + 9)*20**3 + previous//20
        if current not in totals: totals[current] = (secret_n%10, j)
        elif totals[current][1] != j: totals[current] = (totals[current][0]+secret_n%10, j)
        previous = current
    if j%100==0: print(f"{j:4d}/{len(init_secret):4d}", end="\r")
print(f"{j:4d}/{len(init_secret):4d}", end="\r")
max(totals.values()), len(totals), {k:v for k,v in totals.items() if v[0]==2445}

((2445, 2459), 40951, {90989: (2445, 2459)})

Faster by using defaultdict (actually two), and base 19

In [3]:
from collections import defaultdict

In [5]:
prices, totals, last_add = [], defaultdict(int), defaultdict(int)
for j in range(len(init_secret)):
    secret_n = init_secret[j]
    prices.append([secret_n%10])
    previous = 0
    for i in range(1,4):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        current = (prices[-1][-1] - prices[-1][-2] + 9)*19**3 + previous//19
        previous = current
    for i in range(4,2001):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        current = (prices[-1][-1] - prices[-1][-2] + 9)*19**3 + previous//19
        if last_add[current] != j: last_add[current] = j; totals[current] += secret_n%10
        previous = current
    if j%100==0: print(f"{j:4d}/{len(init_secret):4d}", end="\r")
print(f"{j:4d}/{len(init_secret):4d}", end="\r")
max(totals.values()), len(totals), {k:v for k,v in totals.items() if v==2445}

(2445, 40951, {78156: 2445})

Surprisingly, it turns out to be more efficient to generate a huge list of lists name prices, with ~2400x2000 (4.9M) entries than to use only two variables that are overwritten every time

In [4]:
totals, last_add = defaultdict(int), defaultdict(int)
for j in range(len(init_secret)):
    secret_n = init_secret[j]
    prev_price, prev_seq = 0,0
    for i in range(1,4):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        # price = secret_n%10
        diff_seq = (secret_n%10 - prev_price + 9)*19**3 + prev_seq//19
        prev_price, prev_seq = secret_n%10, diff_seq
    for i in range(4,2001):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        # price = secret_n%10
        diff_seq = (secret_n%10 - prev_price + 9)*19**3 + prev_seq//19
        # if diff_seq not in totals: totals[diff_seq] = (price, j)
        # elif totals[diff_seq][1] != j: totals[diff_seq] = (totals[diff_seq][0]+price, j)
        if last_add[diff_seq] != j: last_add[diff_seq] = j; totals[diff_seq] += secret_n%10
        # if totals[diff_seq][1] != j: totals[diff_seq] = (totals[diff_seq][0]+price, j)
        prev_price, prev_seq = secret_n%10, diff_seq
    if j%100==0: print(f"{j:4d}/{len(init_secret):4d}", end="\r")
print(f"{j:4d}/{len(init_secret):4d}", end="\r")
max(totals.values()), len(totals), {k:v for k,v in totals.items() if v==2445}

(2445, 40951, {78156: 2445})

Most readable solution: pricediffs as a dictionary of dictionaries, the inner dict indexed by the tuples of 4 price differences. ~39s (slow to build dict)

In [189]:
prices, pricediffs = [], []
for j in range(len(init_secret)):
    secret_n = init_secret[j]
    prices.append([secret_n%10])
    for i in range(1,4):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
    pricediffs.append({})
    for i in range(4,2001):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        thetuple = tuple(prices[-1][ii-4] - prices[-1][ii-5] for ii in range(4))
        if thetuple not in pricediffs[-1]:
            pricediffs[-1].update({thetuple: i})
        
cand_sequ = {dic_key for pr_df_dic in pricediffs for dic_key in pr_df_dic.keys() if dic_key[3]>0}
len(cand_sequ)

18756

In [190]:
maxsale, best_seq = 0, []
while cand_sequ:
    sale, sequence = 0, cand_sequ.pop()
    for j in range(len(prices)):
        if sequence in pricediffs[j]:
            sale += prices[j][pricediffs[j][sequence]]
    if len(cand_sequ)%100==0: print(f"{len(cand_sequ):5d}", end="\r")
    if sale > maxsale:
        print(f"{len(cand_sequ):5d}", sequence, sale, end="            \r")
        maxsale = sale
        best_seq = sequence
best_seq, maxsale

((0, 0, -2, 2), 2445)

Very efficient solution: encode four-sequences as four-digit base-20 numbers, then as above: pricediffs as a dictionary of dictionaries, the inner dict indexed by the encoded sequences. ~30s total

In [191]:
prices, pricediffs = [], {}
for j in range(len(init_secret)):
    pricediffs[j] = {}
    secret_n = init_secret[j]
    prices.append([secret_n%10])
    previous = 0
    for i in range(1,4):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        current = (prices[-1][-1] - prices[-1][-2] + 9)
        previous = (previous + current*20**3)//20
    for i in range(4,2001):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
        current = (prices[-1][-1] - prices[-1][-2] + 9)
        if current*20**3 + previous not in pricediffs[j]:
            pricediffs[j].update({current*20**3 + previous : i})
        previous = (previous + current*20**3)//20


In [192]:
cand_sequ = {k for pr_df_dic in pricediffs.values() for k in pr_df_dic.keys()}
cand_sequ = {pr_df for pr_df in cand_sequ  if pr_df >= 10*20**3}
len(cand_sequ)

18756

In [193]:
maxsale, best_seq = 0, []
while cand_sequ:
    sale, sequence = 0, cand_sequ.pop()
    for j in range(len(prices)):
        if sequence in pricediffs[j]:
            sale += prices[j][pricediffs[j][sequence]]
    if len(cand_sequ)%100==0: print(f"{len(cand_sequ):5d}", end="\r")
    if sale > maxsale:
        print(f"{len(cand_sequ):5d}", sequence, sale, end="            \r")
        maxsale = sale
        best_seq = sequence
best_seq, maxsale

(90989, 2445)

Trash

In [ ]:
#Using string search. The first acceptable one, but still a bit long (~4min total)
prices = []
for secret_n in init_secret:
    prices.append([secret_n%10])
    for i in range(2000):
        secret_n = ((64*secret_n)^secret_n) % 16777216
        secret_n = ((secret_n//32)^secret_n) % 16777216
        secret_n = ((2048*secret_n)^secret_n) % 16777216
        prices[-1].append(secret_n%10)
pricediffs = ["".join([f"{lst[i+1]-lst[i]:+d}" for i in range(len(lst)-1)]) for lst in prices] #stringformat

In [ ]:
cand_sequ = {pr_df_lst[i:i+8] for pr_df_lst in pricediffs for i in range(0,len(pr_df_lst)-6,2) if pr_df_lst[i+6]=="+"}
cand_sequ = {sequ for sequ in cand_sequ if sequ[-1]!="0"}
len(cand_sequ)

18756

In [ ]:
maxsale, best_seq = 0, []
while cand_sequ:
    sale, sequence = 0, cand_sequ.pop()
    # max_from_seq = min(9, 9 + sum(int(sequence[i:i+2]) for i in range(0,8,2)))
    for j in range(len(prices)):
        if sequence in pricediffs[j]:
            i = pricediffs[j].index(sequence)
            sale += prices[j][i//2+4]
    # if (len(prices)-j) * max_from_seq + sale < maxsale: break #no chance of reaching current max (little benefit)
    if len(cand_sequ)%100==0: print(f"{len(cand_sequ):5d}", end="\r")
    if sale > maxsale:
        print(f"{len(cand_sequ):5d}", sequence, sale, end="            \r")
        maxsale = sale
        best_seq = sequence
best_seq, maxsale

('+0+0-2+2', 2445)

In [ ]:
#conceptually ok, but takes too long. The string search is much faster


pricediffs = [[lst[i+1]-lst[i] for i in range(len(lst)-1)] for lst in prices]
cand_sequ = {tuple(pr_df_lst[i:i+4]) for pr_df_lst in pricediffs for i in range(len(pr_df_lst)-3) if pr_df_lst[i+3]>0}
cand_sequ = {sequ for sequ in cand_sequ if sum(sequ)>0}

maxsale, best_seq = 0, []
while cand_sequ:
    sale, sequence = 0, list(cand_sequ.pop())
    max_from_seq = min(9, 9 + sum(sequence))
    for j in range(len(prices)):
        for i in range(len(pricediffs[j])-4):
            if pricediffs[j][i] == sequence[0] and pricediffs[j][i:i+4] == sequence:
                sale += prices[j][i+4]
                break
        if (len(prices)-j) * max_from_seq + sale < maxsale: break #no chance of reaching current max
    if len(cand_sequ)%100==0: print(f"{len(cand_sequ):5d}", end="\r")
    if sale > maxsale:
        print(f"{len(cand_sequ):5d}", sequence, sale, end="            \r")
        maxsale = sale
        best_seq = sequence
best_seq, maxsale

In [153]:
#way too slow
maxsale, best_seq = 0, []
stored_sales = {sequence: 0 for sequence in cand_sequ}
for j in range(len(prices)):
    pricedifflst = pricediffs[j]
    for sequence in cand_sequ:
        # sale, sequence = 0, cand_sequ.pop()
        if sequence in pricedifflst:
            i = pricedifflst.index(sequence)
            stored_sales[sequence] += prices[j][i//2+4]
    if j%100==0: print(f"{j:5d}", end="\r")
{k:v for k,v in stored_sales.items() if v == max(stored_sales.values())}

numpy solution slow...

In [ ]:
prices = np.array([n%10 for n in init_secret], dtype=np.int32).reshape(-1,1)
secret_n = np.array(init_secret, dtype=np.int32).reshape(-1,1)
for i in range(2000):
    secret_n = ((64*secret_n)^secret_n) % 16777216
    secret_n = ((secret_n//32)^secret_n) % 16777216
    secret_n = ((2048*secret_n)^secret_n) % 16777216
    prices = np.hstack((prices,secret_n%10))

In [ ]:
pricediffs = prices[:,1:] - prices[:,:-1] + 9 #encoding the differences from 0 to 18, then base 20
all_seqs = pricediffs[:,:-3] + 20*pricediffs[:,1:-2] + 20**2*pricediffs[:,2:-1] + 20**3*pricediffs[:,3:] #all sequences as a base-20 sum
all_unique_seqs = np.unique(all_seqs, return_index=False)
all_unique_seqs = {seq for seq in all_unique_seqs if seq>=10*20**3}
len(all_unique_seqs), max(all_unique_seqs)

(18756, 147789)

In [159]:
maxsale = 0
# for seq in all_unique_seqs: #[7+10*20+8*20**2+12*20**3]: #[9+9*20+7*20**2+11*20**3]:#
while all_unique_seqs:
    sale, seq = 0, all_unique_seqs.pop()
    # seq_locs = np.argwhere(all_seqs == seq)
    for row, col in np.argwhere(all_seqs==seq):
    # for row, col in {loc[0]:loc[1] for loc in np.argwhere(all_seqs == seq)[::-1]}.items():
        sale += prices[row, col+4]
    if len(all_unique_seqs)%100==0: print(f"{len(all_unique_seqs):6d}", end="\r")
    if sale > maxsale:
        print(f"{len(all_unique_seqs):6d}", seq, sale, end="            \r")
        maxsale = sale
        best_seq = seq

In [7]:
# looking for a 9-only solution... trash
candidates = []
for j in range(len(prices)//200):
    cands_for_this = []
    for i in range(4,len(prices[j])):
        if prices[j][i]==9:
            if pricediffs[j][i-4:i] not in pricediffs[j][:i-1]:
                cands_for_this.append(pricediffs[j][i-4:i])
    candidates.append(cands_for_this)

In [8]:
viable_cands = []
for cand in candidates[4]: #shortest
    if {cand in cands for cands in candidates}=={True}:
        viable_cands.append(cand)
viable_cands

[]